In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../"))

In [3]:
import numpy as np
np.set_printoptions(linewidth=np.inf, precision=4, suppress=True, threshold=sys.maxsize)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import functools
import scipy

In [ ]:
import desc

from desc.basis import *
from desc.backend import *
from desc.compute import *
from desc.coils import *
from desc.equilibrium import *
from desc.examples import *
from desc.grid import *
from desc.geometry import *

from desc.objectives import *
from desc.objectives.objective_funs import *
from desc.objectives.getters import *
from desc.objectives.normalization import *
from desc.objectives.utils import *
from desc.optimize._constraint_wrappers import *

from desc.transform import Transform
from desc.plotting import *
from desc.optimize import *
from desc.perturbations import *
from desc.profiles import *
from desc.compat import *
from desc.utils import *
from desc.magnetic_fields import *

from desc.__main__ import *
from desc.vmec_utils import *
from desc.input_reader import *
from desc.continuation import *

print_backend_info()

DESC version=0.13.0+1698.gf12602085.
Using JAX backend: jax version=0.5.0, jaxlib version=0.5.0, dtype=float64.
Using device: CPU, with 7.73 GB available memory.


In [5]:
eq = get("HELIOTRON")
eq.R_lmn.shape

(598,)

In [5]:
method = "direct1"
for L in [18, 20, 25, 30, 36, 54, 72, 90]:
    M = eq.M_grid
    N = eq.N_grid
    grid = ConcentricGrid(L=L, M=M, N=N, NFP=eq.NFP, sym=eq.sym)
    transform = Transform(grid=grid, basis=eq.R_basis, method=method)
    if method in ["direct2"]:
        A = transform.matrices[method][0]
    elif method in ["fft"]:
        A = transform.matrices[method][0][0]
    else:
        A = transform.matrices[method][0][0][0]
    cond = np.linalg.cond(A)
    print(f"{cond:.2e}")

5.25e+18
9.55e+17
5.99e+04
1.79e+01
1.19e+01
1.20e+01
1.21e+01
1.22e+01


In [6]:
A.shape

(5980, 598)

In [ ]:
grid = LinearGrid(L=1, M=4, N=4)
obj = ObjectiveFunction(ForceBalance(eq))
cons = get_fixed_boundary_constraints(eq)
cons = maybe_add_self_consistency(eq, cons)
con = ObjectiveFunction(cons)
obj = LinearConstraintProjection(obj, con)
obj.build()

Building objective: force
Precomputing transforms
Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed Psi
Building objective: fixed pressure
Building objective: fixed iota
Building objective: fixed sheet current
Building objective: self_consistency R
Building objective: self_consistency Z
Building objective: lambda gauge
Building objective: axis R self consistency
Building objective: axis Z self consistency


In [8]:
J = obj.jac_scaled_error(obj.x())

In [9]:
cond = np.linalg.cond(J)
print(f"{cond:.2e}")

2.23e+07
